In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import warnings

In [27]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# Load Dataset

In [28]:
train_data_df = pd.read_csv('train_data.csv')
train_data_df.head()

train_data_df.info()

train_label_df = pd.read_csv("train_label.csv")
train_label_df.head()

train_label_df = pd.read_csv("train_label.csv")
train_label_df.head()

test_data_df = pd.read_csv('test_data.csv') 
test_data_df.head()

test_label_df = pd.read_csv("sample_submission.csv")
test_label_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32000 entries, 0 to 31999
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Source IP Address       32000 non-null  object
 1   Destination IP Address  32000 non-null  object
 2   Source Port             32000 non-null  int64 
 3   Destination Port        32000 non-null  int64 
 4   Protocol                32000 non-null  object
 5   Packet Type             32000 non-null  object
 6   Traffic Type            32000 non-null  object
 7   Payload Data            32000 non-null  object
 8   Malware Indicators      32000 non-null  object
 9   Alerts/Warnings         32000 non-null  object
 10  Attack Signature        32000 non-null  object
 11  Action Taken            32000 non-null  object
 12  Severity Level          32000 non-null  object
 13  User Information        32000 non-null  object
 14  Device Information      32000 non-null  object
 15  Ne

,Anomaly Scores,Id
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4


## 데이터 전처리


In [49]:
# 라벨 병합
train = train_data_df.merge(train_label_df, on="Id")
test = test_data_df.copy()

# 불필요 제거
drop_cols = ["Id"]
train.drop(columns=drop_cols, inplace=True)
test.drop(columns=drop_cols, inplace=True)

#컬럼 그룹 정의
target = "Anomaly Scores"

numeric_cols = [
    "Source Port",
    "Destination Port",
    "Packet Length"
]

high_cardinality_cols = [
    "Source IP Address",
    "Destination IP Address",
    "Attack Signature"
]

categorical_cols = [
    "Protocol",
    "Packet Type",
    "Traffic Type",
    "Malware Indicators",
    "Alerts/Warnings",
    "Action Taken",
    "Severity Level",
    "User Information",
    "Device Information",
    "Network Segment",
    "Geo-location Data",
    "Log Source"
]

# 결측치 처리
# 수치형
for col in numeric_cols:
    train[col] = train[col].fillna(train[col].median())
    test[col] = test[col].fillna(train[col].median())

# 문자열
for col in categorical_cols + high_cardinality_cols:
    train[col] = train[col].fillna("Unknown")
    test[col] = test[col].fillna("Unknown")
# 문자열 결측 → "Unknown"이라는 하나의 범주로 유지
# 수치 결측 → 중앙값/(-1) + 필요 시 is_missing 플래그 가 낫다는 gpt의 판단


#로그 변환 / Packet Length에 log1p 적용
train["Packet_Length_log"] = np.log1p(train["Packet Length"])
test["Packet_Length_log"] = np.log1p(test["Packet Length"])

train.drop(columns=["Packet Length"], inplace=True)
test.drop(columns=["Packet Length"], inplace=True)

# 빈도 기반 인코딩 IP, Signature
def frequency_encoding(train, test, col):
    freq = train[col].value_counts()
    train[col + "_freq"] = train[col].map(freq)
    test[col + "_freq"] = test[col].map(freq)
    test[col + "_freq"] = test[col + "_freq"].fillna(0)
    return train, test

for col in high_cardinality_cols:
    train, test = frequency_encoding(train, test, col)


train.drop(columns=high_cardinality_cols, inplace=True)
test.drop(columns=high_cardinality_cols, inplace=True)


# Severity, Actioin 규칙 기반 수치화(SOC 도메인 반영)
severity_map = {
    "Low": 0,
    "Medium": 1,
    "High": 2,
    "Critical": 3
}

#Severity_num == Severity Level 수치화
train["Severity_num"] = train["Severity Level"].map(severity_map).fillna(0)
test["Severity_num"] = test["Severity Level"].map(severity_map).fillna(0)

train["Action_Block"] = (train["Action Taken"] == "Block").astype(int)
test["Action_Block"] = (test["Action Taken"] == "Block").astype(int)


# 타깃 인코딩
def target_encoding(train, test, col, target):
    means = train.groupby(col)[target].mean()
    train[col + "_te"] = train[col].map(means)
    test[col + "_te"] = test[col].map(means)
    global_mean = train[target].mean()
    test[col + "_te"] = test[col + "_te"].fillna(global_mean)
    return train, test

te_cols = [
    "Protocol",
    "Packet Type",
    "Traffic Type",
    "Alerts/Warnings",
    "Malware Indicators",
    "Network Segment",
    "Log Source"
]

for col in te_cols:
    train, test = target_encoding(train, test, col, target)

train.drop(columns=te_cols, inplace=True)
test.drop(columns=te_cols, inplace=True)


# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder

le_cols = train.select_dtypes(include="object").columns

for col in le_cols:
    le = LabelEncoder()
    all_data = pd.concat([train[col], test[col]])
    le.fit(all_data)
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])

test



,Source Port,Destination Port,Payload Data,Action Taken,Severity Level,User Information,Device Information,Geo-location Data,Packet_Length_log,Source IP Address_freq,...,Attack Signature_freq,Severity_num,Action_Block,Protocol_te,Packet Type_te,Traffic Type_te,Alerts/Warnings_te,Malware Indicators_te,Network Segment_te,Log Source_te
0,4999,32694,31529,1,1,2382,10191,3835,7.109062,0.0,...,15919,0,0,50.103268,50.225172,50.279746,50.022482,50.086315,50.070164,50.145526
1,55643,11859,37957,1,1,19918,14001,8702,6.990257,0.0,...,16081,0,0,50.203893,50.225172,50.279746,50.022482,50.046750,49.996124,50.145526
2,5855,4424,31383,2,2,11472,18557,833,7.210080,0.0,...,15919,1,0,50.203893,50.225172,50.279746,50.110937,50.086315,49.996124,50.145526
3,65513,21972,38031,0,2,28142,489,294,7.170888,0.0,...,16081,1,0,50.103268,49.902421,50.418664,50.022482,50.086315,49.996124,50.145526
4,36878,7510,15350,0,2,26708,17385,5127,5.638355,0.0,...,16081,1,0,50.203893,49.902421,50.418664,50.022482,50.086315,50.133226,49.986710
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6410,42408,28746,8145,2,0,6133,18511,3996,7.252054,0.0,...,15919,2,0,50.203893,50.225172,49.501079,50.110937,50.046750,50.070164,50.145526
6411,62459,26655,3718,2,1,28505,11552,4787,5.996452,0.0,...,15919,0,0,50.203893,49.902421,49.501079,50.022482,50.086315,49.996124,49.986710
6412,35056,37119,26290,2,0,13027,29126,2640,7.017506,0.0,...,16081,2,0,50.103268,50.225172,50.279746,50.110937,50.046750,50.070164,50.145526
6413,36362,11584,4177,0,2,8477,17245,1192,7.036148,0.0,...,15919,1,0,49.893638,50.225172,50.279746,50.022482,50.086315,50.070164,49.986710


In [30]:
# 데이터 분리
X_train = train.drop(columns=[target])
y_train = train[target]
X_test = test


print(X_train.shape, X_test.shape)


(32000, 21) (6415, 21)


# Modeling

In [31]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_curve, auc

In [33]:
random_state = 42  # 실험결과 재현을 위해 random_state 고정

In [34]:
categorical_features = ['Source IP Address', 'Destination IP Address', 'Source Port',
                        'Destination Port', 'Protocol', 'Packet Type',
                        'Traffic Type', 'Payload Data', 'Malware Indicators', 'Alerts/Warnings',
                        'Attack Signature', 'Action Taken',
                        'Severity Level', 'User Information', 'Device Information',
                        'Network Segment', 'Geo-location Data', 'Log Source']

numerical_features = ['Packet Length']

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

X = train_data_df[categorical_features + numerical_features]
y = train_label_df['Anomaly Scores']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=random_state)

numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ])

In [35]:
X_train_processed = preprocessor.fit_transform(X_train)
X_val_processed = preprocessor.transform(X_val)

In [36]:
target_scaler = StandardScaler()
y_train = y_train.values
y_train = y_train.reshape(-1, 1)
y_val = y_val.values
y_val = y_val.reshape(-1, 1)

y_train_scaled = target_scaler.fit_transform(y_train)
y_val_scaled = target_scaler.transform(y_val)

In [37]:
lr = LinearRegression()

In [38]:
lr.fit(X_train_processed, y_train_scaled.ravel())

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [39]:
train_pred_scaled = lr.predict(X_train_processed)
train_pred = target_scaler.inverse_transform(train_pred_scaled.reshape(-1, 1))

In [40]:
val_pred_scaled = lr.predict(X_val_processed)
val_pred = target_scaler.inverse_transform(val_pred_scaled.reshape(-1, 1))

In [41]:
print(f'Train: RMSE({np.sqrt(mean_squared_error(y_train, train_pred))})')
print(f'Validation: RMSE({np.sqrt(mean_squared_error(y_val, val_pred))})')

Train: RMSE(0.005197746365171589)
Validation: RMSE(29.754539964966902)


# Test Submission

In [42]:
categorical_features = ['Source IP Address', 'Destination IP Address', 'Source Port',
                        'Destination Port', 'Protocol', 'Packet Type',
                        'Traffic Type', 'Payload Data', 'Malware Indicators', 'Alerts/Warnings',
                        'Attack Signature', 'Action Taken',
                        'Severity Level', 'User Information', 'Device Information',
                        'Network Segment', 'Geo-location Data', 'Log Source']

numerical_features = ['Packet Length']

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

X_test = test_data_df[categorical_features + numerical_features]


numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
    ])

In [43]:
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

In [44]:
test_pred_scaled = lr.predict(X_test_processed)
test_pred = target_scaler.inverse_transform(test_pred_scaled.reshape(-1, 1))

In [45]:
test_label_df['Anomaly Scores'] = test_pred

In [46]:
test_label_df

,Anomaly Scores,Id
0,56.643729,0
1,44.218357,1
2,57.613356,2
3,42.919029,3
4,43.594750,4
...,...,...
6410,51.164410,6410
6411,67.268572,6411
6412,52.477637,6412
6413,56.790131,6413


In [47]:
test_label_df.to_csv('test_submission.csv', index=False)